In [243]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.core.internals.construction import to_arrays
from prompt_toolkit.utils import to_int
from pulp import *

In [244]:
def find_loops(edges):
    """
    Find all closed loops starting and ending at node 0 in a directed graph.

    Args:
        edges: List of tuples representing directed edges (from_node, to_node)

    Returns:
        List of lists, where each sublist contains tuples forming a closed loop from 0 to 0
    """
    # Create adjacency list for faster lookup
    adj = {}
    for start, end in edges:
        if start not in adj:
            adj[start] = []
        adj[start].append(end)

    def find_paths(current, path, visited):
        """
        Recursive helper function to find all paths from current node back to 0.
        """
        if current == 0 and len(path) > 0:  # Found a loop back to 0
            return [path[:]]

        if current not in adj:  # Dead end
            return []

        paths = []
        for next_node in adj[current]:
            if (current, next_node) not in visited:  # Avoid using same edge twice
                path.append((current, next_node))
                visited.add((current, next_node))
                paths.extend(find_paths(next_node, path, visited))
                path.pop()
                visited.remove((current, next_node))
        return paths

    # Start DFS from node 0
    all_loops = find_paths(0, [], set())

    # Filter out non-minimal loops (loops that contain other loops)
    minimal_loops = []
    for loop in all_loops:
        # Check if this loop contains any other loop
        is_minimal = True
        visited_nodes = {node for edge in loop for node in edge}
        for other_loop in all_loops:
            if loop != other_loop:
                other_nodes = {node for edge in other_loop for node in edge}
                if other_nodes.issubset(visited_nodes) and len(other_loop) < len(loop):
                    is_minimal = False
                    break
        if is_minimal:
            minimal_loops.append(loop)

    return minimal_loops

## Cost matrix

In [245]:
cost_matrix = pd.read_csv('cost_matrix.csv')

# Convert the entire DataFrame to integers
cost_matrix = cost_matrix.astype(int)

# If the index of the DataFrame needs to be integers (e.g., if the index is non-numeric):
cost_matrix.index = cost_matrix.index.astype(int)

# If the column names need to be integers (if they are non-numeric or string-based):
cost_matrix.columns = cost_matrix.columns.astype(int)

## Shop demands

In [246]:
shop_demands = pd.read_csv('shop_demands.csv').to_numpy(dtype=int)

## Vehicle routing problem

In [247]:
# Vehicle capacity
van_capacity = 300

#Number of shops
n = cost_matrix.shape[0]

#Set N of shops
N = [i for i in range(1,n)]

#Set of shops and storage
V = [0] + N

In [248]:
# Creates a list of tuples containing all the possible routes for transport
A=[(i,j) for i in V for j in V if i!=j]

In [249]:
# A set of variables x is created to contain the vehicle routes variables (binary var)
x = {}
for a in A:
    x[a] = LpVariable("x(%s,%s)" %a, cat=LpBinary)

# A set of variables u is created to contain the continuous delivered quantity
u = {}
for i in N:
    u[i] = LpVariable("u(%s)" %i, shop_demands[i], van_capacity)

In [250]:
# Creates the 'prob' variable to contain the problem data
prob = LpProblem("CVRP_Problem",LpMinimize)

In [251]:
# The objective function is added to 'prob' first
prob += lpSum(cost_matrix.iloc[i,j]*x[i,j] for (i,j) in A)

In [252]:
#Constraint 1 : imposes that exactly one arc enters each customer node
one_visit_out = {}
for i in N:
    one_visit_out[i] = lpSum([x[i,j] for j in V if j!=i]) == 1.0
    prob += one_visit_out[i]

In [253]:
#Constraint 2 : imposes that exactly one arc leaves each customer node
one_visit_in = {}
for j in N:
    one_visit_in[j] = lpSum([x[i,j] for i in V if i!=j]) == 1.0
    prob += one_visit_in[j]

In [254]:
#Constraint 4 : imposes the amount of flow that leaves the depot should be identical
#with the flow returns to the depot
depot_node = lpSum([x[0,j] for j in N]) == lpSum([x[i,0] for i in N])
prob += depot_node

In [255]:
#Constraint 5 : vehicle capacity constraint and sub tour elimination constraint
sub_tours_cap = {}
for i in N:
    for j in N:
        if i != j:
            sub_tours_cap[i,j] = u[i] - u[j] + van_capacity * x[i,j] <= van_capacity - shop_demands[j]
            prob += sub_tours_cap[i,j]

In [256]:
# The problem is solved using PuLP's choice of Solver (ici Cplex, retourne 1 si résolu, -1 sinon)
prob.solve(GUROBI(msg=False))

1

In [257]:
# The status of the solution is printed to the screen (optimal ou pas)
print("Status:", LpStatus[prob.status])

Status: Optimal


In [258]:
# Each of the variables is printed with it's resolved optimum value
for key,val in x.items():
    if val.varValue==1.0:
        print(val,"=",val.varValue)

x(0,2) = 1.0
x(0,4) = 1.0
x(1,0) = 1.0
x(2,3) = 1.0
x(3,0) = 1.0
x(4,1) = 1.0


In [259]:
# The optimised objective function value is printed to the screen
print("Total Cost = ", value(prob.objective))

Total Cost =  10.0


In [260]:
# Afficher les acrs correspondant à la solution
active_arcs = [key for key,val in x.items() if val.varValue >=0.9 ]
find_loops(active_arcs)


[[(0, 2), (2, 3), (3, 0)], [(0, 4), (4, 1), (1, 0)]]

In [261]:
# Each of the variables is printed with it's resolved optimum value
for key,val in u.items():
    print(val,"=",val.varValue)

u(1) = 160.0
u(2) = 80.0
u(3) = 230.0
u(4) = 120.0
